# Visa Itenerary

Visa Itinerary is a suite of predictive machine learning scores related to a cardholder’s travel behavior that can help you provide marketing campaigns to cardholders already flagged by the travel predict API. This add-on will allow you to provide a “route” at the cardholders' travel destination with geographical locations and/or merchant categories that they are most likely to spend at. This will allow you to guide travelers to merchants they might be interested in and provide incentives to purchase goods and services there. 

In [2]:
import folium
import pandas as pd
import csv
import random

## Functions 
Function for heatmap

In [4]:
def get_color(count, metric):
    if metric == 'volume':
        if count < 11:
            return "lightgray"
        elif count < 21:
            return "gray"
        elif count < 31:
            return "lightred"
        elif count < 41:
            return "red"
        elif count < 51:
            return "darkred"
        elif count > 50:
            return "black"
    elif metric == 'amount':
        if count < 26:
            return "lightgray"
        elif count < 51:
            return "gray"
        elif count < 76:
            return "lightred"
        elif count < 101:
            return "red"
        elif count < 999:
            return "darkred"
        elif count > 999:
            return "black"

## Pulling Transaction Data

In [5]:
data = pd.read_csv("TransactionMockDataBig.csv")
groups = data.groupby('Merchant_Name')
counts = groups.size().reset_index(name='counts')
amount = groups['Transaction_Amount'].sum().reset_index(name='sum')

data.head()

,Transaction_Time,Merchant_Name,Location,Category,Transaction_Amount,IDs
0,2020-10-30 19:7:51,Cultivar San Francisco,"37.799820, -122.442413",Restaurant,18.93,40
1,2020-10-29 1:18:9,Tartine Manufactroy,"37.762030,-122.411600",Restaurant,10.24,266
2,2020-10-29 18:24:43,Cultivar San Francisco,"37.799820, -122.442413",Restaurant,27.61,619
3,2020-10-30 14:17:34,The Argentum Project,"37.781689, -122.409218",Restaurant,3.50,893
4,2020-10-31 3:48:55,Tartine Manufactroy,"37.762030,-122.411600",Restaurant,1.85,385


## "Heat" Map by Transaction Volume

In [6]:
# Uses lat then lon. The bigger the zoom number, the closer in you get
bay_area_map_volume = folium.Map(location=[37.7577627, -122.4727053,], zoom_start = 12.5, tiles='Stamen Terrain')

tooltip = 'Click to see Merchant'

for row in groups.groups:
    trans = groups.get_group(row).iloc[:1]
    coord = trans.Location.values[0].split(",")
    merch = '<i>' + trans.Merchant_Name.values[0] + '</i>'
    num_trans = counts.loc[counts['Merchant_Name'] == trans.Merchant_Name.values[0]].counts
    color = get_color(num_trans.values[0], 'volume')
    
    folium.Marker(
        [float(coord[0]), float(coord[1])], 
        popup=merch, 
        tooltip=tooltip,
        icon=folium.Icon(color=color),
        ).add_to(bay_area_map_volume)

bay_area_map_volume # Calls the map to display

## "Heat" Map by Transaction Amount

In [7]:
# Uses lat then lon. The bigger the zoom number, the closer in you get
bay_area_map_amount = folium.Map(location=[37.7577627, -122.4727053,], zoom_start = 12.5, tiles='Stamen Terrain')

tooltip = 'Click to see Merchant'

for row in groups.groups:
    trans = groups.get_group(row).iloc[:1]
    coord = trans.Location.values[0].split(",")
    merch = '<i>' + trans.Merchant_Name.values[0] + '</i>'
    trans_sum = amount.loc[amount['Merchant_Name'] == trans.Merchant_Name.values[0]]['sum']
    color = get_color(trans_sum.values[0], 'amount')
    
    folium.Marker(
        [float(coord[0]), float(coord[1])], 
        popup=merch, 
        tooltip=tooltip,
        icon=folium.Icon(color=color),
        ).add_to(bay_area_map_amount)

bay_area_map_amount # Calls the map to display

## Create Itenerary

In [24]:
#Opens up the Transaction CSV to be read
with open('TransactionMockDataBig.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    #Array to store recommendations and send to issuer
    iteneraryArray = []
    #Reading each row
    for row in readCSV:
        category = row[3]
        merchant_name = row[1]
        location = row[2]

        if row[3] != "Category":
            iteneraryArray.append([category,merchant_name,location])

    #remove duplicates for testing purposes	
    duplicateMerchant = []
    output = []
    for x in iteneraryArray:
        if x[1] not in duplicateMerchant:
            duplicateMerchant.append(x[1])
            output.append(x)

#selecting 3 different options
sampling = random.sample(output, k=3)
sampling.insert(0,["Category","Merchant_name","Location"]) 

#Opens the Iternerary CSV to send to issuers
with open('IterneraryRecommendation.csv','w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerows(sampling)
    
recs = pd.read_csv("IterneraryRecommendation.csv")
recs.head(3)

,Category,Merchant_name,Location
0,Restaurant,Presidio Kebab,"37.7881, -122.4468"
1,Restaurant,Marufuku Ramen SF,"37.782100, -122.403770"
2,Restaurant,Kokkari Estiatorio,"37.7970, -122.3997"


## Recommendation Map

In [27]:
# Uses lat then lon. The bigger the zoom number, the closer in you get
rec_map = folium.Map(location=[37.7577627, -122.4727053,], zoom_start = 12.5, tiles='Stamen Terrain')

tooltip = 'Click to see Merchant'

for index, row in recs.iloc[0:].iterrows():
    coord = row.Location.split(",")
    merch = '<i>' + row.Merchant_name + '</i>'
    color = "green"
    
    folium.Marker(
        [float(coord[0]), float(coord[1])], 
        popup=merch, 
        tooltip=tooltip,
        icon=folium.Icon(color=color),
        ).add_to(rec_map)

rec_map # Calls the map to display